In [319]:
pip install matplotlib


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 19.2 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 300.4/300.4 kB 34.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 25.4 MB/s eta 0:00:00m eta 0:00:010:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 38.2 MB/s eta 0:00:00m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 42.8 MB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 15.0 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.0 -> 23.2.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [1]:
'ALA', 'VAL', 'ILE', 'LEU', 'MET', 'PHE', 'TRP', 'GLU', 'TYR', 'ASP', 'ASN', 'GLN', 'ARG', 'LYS', 'SER', 'THR', 'CYS', 'HIS'



In [13]:
import os
os.chdir('/home/vlad/Desktop/Sucessful_Osprey_February/result')
os.getcwd()

'/home/vlad/Desktop/Sucessful_Osprey_February/result'

In [ ]:
#this script sort keys by the distances for each object (A and B) and add variables to json


import os
import json

def add_variables_to_json(json_file, variables):
    # Read the JSON data from the file
    with open(json_file, 'r') as f:
        data = json.load(f)
        
    # Add the Variables object to the JSON
    data["Complex_Size"] = variables

    # Sort the keys inside each object (A and B) based on their values
    for key in data:
        if "A" in data[key]:
            data[key]["A"] = {k: v for k, v in sorted(data[key]["A"].items(), key=lambda item: item[1])}
        if "B" in data[key]:
            data[key]["B"] = {k: v for k, v in sorted(data[key]["B"].items(), key=lambda item: item[1])}
        
    # Save the modified JSON with the same name
    with open(json_file, 'w') as f:
        json.dump(data, f, indent=4)

# List of JSON files (replace 'file1.json', 'file2.json', and 'file3.json' with your actual file names)
json_files = ['out_2bcx.json', 'out_6xxf.json', 'out_6y4o.json']

# Define the variables to be added at the end of each JSON
variables = {
    "mutant_beginning": "beginning",
    "mutant_ending": "ending",
    "ligand_beginning": "beginning",
    "ligand_ending": "ending"
}

# Loop through the JSON files, add the Variables object, and save with the same names
for json_file in json_files:
    add_variables_to_json(json_file, variables)


In [320]:
# this script reads jsons and creates python scripts for Osprey in individual folder

import os
import json

def create_folders_and_files(json_file):
    # Extract the name of the JSON file (without the extension) to be used as the folder name
    file_name = os.path.splitext(os.path.basename(json_file))[0]

    # Create a folder with the name of the JSON file (if it doesn't exist)
    if not os.path.exists(file_name):
        os.mkdir(file_name)

    # Extract the part of the JSON file name that comes after "out_"
    word1 = file_name.split("out_")[-1]

    # Read the JSON data from the file
    with open(json_file, 'r') as f:
        data = json.load(f)

    # Check if the key "Complex_Size" exists in the JSON data
    if "Complex_Size" in data:
        # Extract the variables from the "Complex_Size" dictionary
        variables = data["Complex_Size"]
        variable1 = variables.get("mutant_beginning", "KEY1")
        variable2 = variables.get("mutant_ending", "KEY2")
        variable3 = variables.get("ligand_beginning", "KEY3")
        variable4 = variables.get("ligand_ending", "KEY4")
        
    else:
        # If "Complex_Size" key is not present, set default values for variables
        variable1 = "KEY1"
        variable2 = "KEY2"
        variable3 = "KEY3"
        variable4 = "KEY4"
        print("Complex_Size key not found. Using default values for variables may affect osprey calculations. Please, check your .json file")
        
    # Loop through the JSON data and create folders and .py files
    for key, subdata in data.items():
        
        # Skip processing if the key is "Complex_Size"
        if key == "Complex_Size":
            continue

        # Remove the last character from the key
        truncated_key = key[:-1]
    
        # Create a folder with the name of the key (if it doesn't exist)
        subfolder = os.path.join(file_name, key)
        if not os.path.exists(subfolder):
            os.mkdir(subfolder)

        # Get the subkeys for object "A" and "B"
        subkeys_from_A = list(subdata["A"].keys())
        subkeys_from_B = list(subdata["B"].keys())            
            
        # Create the content for the .py file
        py_content = f'''

import osprey
osprey.start(heapSizeMiB=100000, garbageSizeMiB=8192)

# choose a forcefield
ffparams = osprey.ForcefieldParams()

# read a PDB file for molecular info
mol = osprey.readPdb('/home/vlad/Desktop/Sucessful_Osprey_February/{word1}_reduce.pdb')

# make sure all strands share the same template library
templateLib = osprey.TemplateLibrary(ffparams.forcefld)

# define the protein strand
protein = osprey.Strand(mol, templateLib=templateLib, residues=['{variable1}', '{variable2}'])
protein.flexibility['{truncated_key}'].setLibraryRotamers(osprey.WILD_TYPE, 'ALA', 'VAL', 'ILE', 'LEU', 'MET', 'PHE', 'TRP', 'GLU', 'TYR', 'ASP', 'ASN', 'GLN', 'ARG', 'LYS', 'SER', 'THR', 'CYS', 'HIS').addWildTypeRotamers().setContinuous()

# define the ligand strand
ligand = osprey.Strand(mol, templateLib=templateLib, residues=['{variable3}', '{variable4}'])
'''

        # Add subkeys from object "A" to the .py content
        for subkey_A in subkeys_from_A:
            py_content += f"protein.flexibility['{subkey_A}'].setLibraryRotamers(osprey.WILD_TYPE).addWildTypeRotamers().setContinuous()\n"

        # Add subkeys from object "B" to the .py content
        for subkey_B in subkeys_from_B:
            py_content += f"ligand.flexibility['{subkey_B}'].setLibraryRotamers(osprey.WILD_TYPE).addWildTypeRotamers().setContinuous()\n"

        py_content += '''
        
        
# make the conf space for the protein
proteinConfSpace = osprey.ConfSpace(protein)

# make the conf space for the ligand
ligandConfSpace = osprey.ConfSpace(ligand)

# make the conf space for the protein+ligand complex
complexConfSpace = osprey.ConfSpace([protein, ligand])

# how should we compute energies of molecules?
# (give the complex conf space to the ecalc since it knows about all the templates and degrees of freedom)
parallelism = osprey.Parallelism(cpuCores=16, gpus=1, streamsPerGpu=82)
minimizingEcalc = osprey.EnergyCalculator(complexConfSpace, ffparams, parallelism=parallelism, isMinimizing=True)

# BBK* needs a rigid energy calculator too, for multi-sequence bounds on K*
rigidEcalc = osprey.SharedEnergyCalculator(minimizingEcalc, isMinimizing=False)

# how should we define energies of conformations?
def confEcalcFactory(confSpace, ecalc):
	eref = osprey.ReferenceEnergies(confSpace, ecalc)
	return osprey.ConfEnergyCalculator(confSpace, ecalc, referenceEnergies=eref)

# configure BBK*
bbkstar = osprey.BBKStar(
	proteinConfSpace,
	ligandConfSpace,
	complexConfSpace,
	numBestSequences=1, # more sequenses - more computation time
	epsilon=0.01, # you proabably want something more precise in your real designs
	writeSequencesToConsole=True,
	writeSequencesToFile='bbkstar_results_{word1}_{key}.tsv'
)

# configure BBK* inputs for each conf space
for info in bbkstar.confSpaceInfos():

	# how should we define energies of conformations?
	eref = osprey.ReferenceEnergies(info.confSpace, minimizingEcalc)
	info.confEcalcMinimized = osprey.ConfEnergyCalculator(info.confSpace, minimizingEcalc, referenceEnergies=eref)

	# compute the energy matrix
	ematMinimized = osprey.EnergyMatrix(info.confEcalcMinimized, cacheFile='emat.%s.dat' % info.id)

	# how should confs be ordered and searched?
	# (since we're in a loop, need capture variables above by using defaulted arguments)
	def makeAStarMinimized(rcs, emat=ematMinimized):
		return osprey.AStarTraditional(emat, rcs, showProgress=False)
	info.confSearchFactoryMinimized = osprey.BBKStar.ConfSearchFactory(makeAStarMinimized)

	# BBK* needs rigid energies too
	confEcalcRigid = osprey.ConfEnergyCalculatorCopy(info.confEcalcMinimized, rigidEcalc)
	ematRigid = osprey.EnergyMatrix(confEcalcRigid, cacheFile='emat.%s.rigid.dat' % info.id)
	def makeAStarRigid(rcs, emat=ematRigid):
		return osprey.AStarTraditional(emat, rcs, showProgress=False)
	info.confSearchFactoryRigid = osprey.BBKStar.ConfSearchFactory(makeAStarRigid)

	# how should we score each sequence?
	# (since we're in a loop, need capture variables above by using defaulted arguments)
	def makePfunc(rcs, confEcalc=info.confEcalcMinimized, emat=ematMinimized):
		return osprey.PartitionFunction(
			confEcalc,
			osprey.AStarTraditional(emat, rcs, showProgress=False),
			osprey.AStarTraditional(emat, rcs, showProgress=False),
			rcs
		)
	info.pfuncFactory = osprey.KStar.PfuncFactory(makePfunc)

# run BBK*
scoredSequences = bbkstar.run(minimizingEcalc.tasks)

# make a sequence analyzer to look at the results
analyzer = osprey.SequenceAnalyzer(bbkstar)

# use results
for scoredSequence in scoredSequences:
	print("result:")
	print("\tsequence: %s" % scoredSequence.sequence)
	print("\tK* score: %s" % scoredSequence.score)

	# write the sequence ensemble, with up to 10 of the lowest-energy conformations
	numConfs = 10
	analysis = analyzer.analyze(scoredSequence.sequence, numConfs)
	print(analysis)
	analysis.writePdb(
		'seq.%s.pdb' % scoredSequence.sequence,
		'Top %d conformations for sequence %s' % (numConfs, scoredSequence.sequence)
	)
'''
        # Replace "{word1}" and "{key}" in the writeSequencesToFile row
        py_content = py_content.replace("{word1}", word1).replace("{key}", key)

        # Write the .py file inside the key's folder
        py_file = os.path.join(subfolder, f'bbkstar_{file_name}_{key}.py')
        with open(py_file, 'w') as f:
            f.write(py_content)

# List of JSON files (replace 'file1.json', 'file2.json', and 'file3.json' with your actual file names)
json_files = ['out_2bcx.json', 'out_6xxf.json', 'out_6y4o.json']

# Loop through the JSON files and create folders and .py files
for json_file in json_files:
    create_folders_and_files(json_file)


In [321]:
# this script runs Osprey and saves the results in a individual folder. RUN IT FROM THE COMMAND LINE!!!

import os
import glob
import subprocess


os.chdir('/home/vlad/Desktop/Sucessful_Osprey_February/result')

# Function to find all files matching the pattern "bbkstar*.py"
def find_bbkstar_files(directory):
    bbkstar_files = []
    for root, dirs, files in os.walk(directory):
        for file in files:
            if file.startswith("bbkstar") and file.endswith(".py"):
                bbkstar_files.append(os.path.join(root, file))
    return bbkstar_files

# Get the current directory
current_directory = os.getcwd()

# Find all "bbkstar*.py" files in the current directory and its subdirectories
bbkstar_files_list = find_bbkstar_files(current_directory)

# Loop through each path, run the Python scripts, and create a txt file with the filename
for path in bbkstar_files_list:
    folder_name = os.path.dirname(path)
    file_name = os.path.splitext(os.path.basename(path))[0]
    
    os.chdir(folder_name)
        
    # Run the Python script using subprocess
    command = f"python3.9 {path}"
    subprocess.run(command, shell=True)
    
#     # Create a txt file with the filename inside the same folder as the Python script
#     with open(os.path.join(folder_name, f"{file_name}.txt"), "w") as f:
#         f.write(file_name)


OSPREY 3.2.213, Python 3.9.17, Java 14.0.1, Linux-5.15.0-76-generic-x86_64-with-glibc2.31
Using up to 100000 MiB heap memory: 8192 MiB for garbage, 91808 MiB for storage
read PDB file from file: /home/vlad/Desktop/Sucessful_Osprey_February/2bcx_reduce.pdb
LEU A   4
LYS B3614
Discovering CUDA GPUs... found 1
GpuStreamPool: using 82 stream(s) across 1 gpu(s)
Calculating reference energies for 228 residue confs...
Progress:  100.0%   ETA: 25.0 ms
Finished in 1.2 s
read energy matrix from file: /home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A72M/emat.protein.dat
read energy matrix from file: /home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A72M/emat.protein.rigid.dat
Calculating reference energies for 8 residue confs...
Progress:  100.0%   ETA: 0 ns
Finished in 6.1 ms
read energy matrix from file: /home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A72M/emat.ligand.dat
read energy matrix from file: /home/vlad/Desktop/Sucessful_Osprey_February/result/out_2

OSPREY 3.2.213, Python 3.9.17, Java 14.0.1, Linux-5.15.0-76-generic-x86_64-with-glibc2.31
Using up to 100000 MiB heap memory: 8192 MiB for garbage, 91808 MiB for storage
read PDB file from file: /home/vlad/Desktop/Sucessful_Osprey_February/2bcx_reduce.pdb
LEU A   4
LYS B3614
Discovering CUDA GPUs... found 1
GpuStreamPool: using 82 stream(s) across 1 gpu(s)
Calculating reference energies for 214 residue confs...
Progress:  100.0%   ETA: 28.0 ms
Finished in 1.1 s
read energy matrix from file: /home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A47E/emat.protein.dat
read energy matrix from file: /home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A47E/emat.protein.rigid.dat
Calculating reference energies for 35 residue confs...
Progress:  100.0%   ETA: 0 ns
Finished in 173.6 ms
read energy matrix from file: /home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A47E/emat.ligand.dat
read energy matrix from file: /home/vlad/Desktop/Sucessful_Osprey_February/result/ou

Traceback (most recent call last):
  File "/home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A15A/bbkstar_out_2bcx_A15A.py", line 67, in <module>
    eref = osprey.ReferenceEnergies(info.confSpace, minimizingEcalc)
  File "/home/vlad/.local/lib/python3.9/site-packages/osprey/__init__.py", line 720, in ReferenceEnergies
    return builder.build()
RuntimeError: java.lang.RuntimeException: can't submit runnable to threads


KeyboardInterrupt: 

In [44]:
bbkstar_files_list

['/home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A72M/bbkstar_out_2bcx_A72M.py',
 '/home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A27I/bbkstar_out_2bcx_A27I.py',
 '/home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A47E/bbkstar_out_2bcx_A47E.py',
 '/home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A15A/bbkstar_out_2bcx_A15A.py',
 '/home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A11E/bbkstar_out_2bcx_A11E.py',
 '/home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A18L/bbkstar_out_2bcx_A18L.py',
 '/home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A32L/bbkstar_out_2bcx_A32L.py',
 '/home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A75K/bbkstar_out_2bcx_A75K.py',
 '/home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A14E/bbkstar_out_2bcx_A14E.py',
 '/home/vlad/Desktop/Sucessful_Osprey_February/result/out_2bcx/A36M/bbkstar_out_2bcx_A36M.py',
 '/home/vlad/Desktop/Sucessful_Osprey_February/res

In [317]:
import os

def read_sequence_column(tsv_file):
    with open(tsv_file, 'r') as file:
        lines = file.readlines()

    # Find the index of the "Sequence" column in the header row
    header_row = lines[1].strip().split('\t')
    sequence_index = header_row.index('Sequence')

    # Extract values from the first row ("data1") and store them in a set to avoid duplicates
    unique_values = set()
    data_row = lines[0].strip().split('\t')  # Read only the first data row
    unique_values.add(data_row[sequence_index])

    return unique_values

def extract_unique_name_fragments(tsv_file, unique_name_fragments_set):
    basename = os.path.basename(tsv_file)
    start_index = basename.find("bbkstar_results_") + len("bbkstar_results_")
    end_index = basename.find("_A")
    unique_name_fragment = basename[start_index:end_index]

    # Check if the unique_name_fragment is already in the set
    if unique_name_fragment not in unique_name_fragments_set:
        unique_name_fragments_set.add(unique_name_fragment)
        return unique_name_fragment
    else:
        return None


def get_tsv_files_paths(base_dir):
    tsv_files = []
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.tsv'):
                tsv_files.append(os.path.join(root, file))
    return tsv_files

if __name__ == "__main__":
    base_directory = "/home/vlad/Desktop/Sucessful_Osprey_February/result"  # Replace this with the actual path to the parent directory
    tsv_files_paths = get_tsv_files_paths(base_directory)

    # Create a set to store unique name fragments
    unique_name_fragments = set()
    
    unique_sequences = set()
    for tsv_file in tsv_files_paths:
        sequences = read_sequence_column(tsv_file)
        unique_sequences.update(sequences)
        
        unique_name_fragment = extract_unique_name_fragments(tsv_file, unique_name_fragments)
        if unique_name_fragment is not None:
            list_of_unique_name_fragments = list(unique_name_fragments)

    # Sort the unique sequences by numerical values (ignoring letters)
    sorted_unique_sequences = sorted(unique_sequences, key=lambda x: int(''.join(filter(str.isdigit, x))))



#     # Print the resulting list
print(list_of_unique_name_fragments)
print(sorted_unique_sequences)



['2bcx', '6y4o', '6xxf']
['A11=glu', 'A14=glu', 'A15=ala', 'A18=leu', 'A19=phe', 'A27=ile', 'A32=leu', 'A35=val', 'A36=met', 'A38=ser', 'A39=leu', 'A41=gln', 'A47=glu', 'A51=met', 'A52=ile', 'A54=glu', 'A55=val', 'A63=ile', 'A68=phe', 'A71=met', 'A72=met', 'A74=arg', 'A75=lys']


In [313]:
import os
import pandas as pd

def read_sequence_column(tsv_file):
    with open(tsv_file, 'r') as file:
        lines = file.readlines()

    # Find the index of the "Sequence" column in the header row
    header_row = lines[1].strip().split('\t')
    sequence_index = header_row.index('Sequence')

    # Extract values from the row with "Seq ID" as the sequence
    row0_data = lines[0].strip().split('\t')
    row2_data = lines[2].strip().split('\t')

    # Find the index of the "K* Score (Log10)" column in the header row
    k_score_index = header_row.index('K* Score (Log10)')

    # Subtract values of row2 from row0 for the "K* Score (Log10)" column
    k_delta = float(row2_data[k_score_index]) - float(row0_data[k_score_index])

    # Get the value of the "Sequence" column from row2
    sequence_wt = row0_data[sequence_index]
    sequence_mut = row2_data[sequence_index].split('=')[1]

#     print(sequence, k_delta)
    return sequence_wt, sequence_mut, k_delta

def get_tsv_files_paths(base_dir):
    tsv_files = []
    for root, _, files in os.walk(base_dir):
        for file in files:
            if file.endswith('.tsv'):
                tsv_files.append(os.path.join(root, file))
    return tsv_files

if __name__ == "__main__":
    base_directory = "/home/vlad/Desktop/Sucessful_Osprey_February/result"
    tsv_files_paths = get_tsv_files_paths(base_directory)

    # ... (existing code to get unique sequences and name fragments) ...

    # Create an empty dictionary to store the data for the DataFrame
    data_dict1 = {'WT_amino_acids': []}
    data_dict2 = {'WT_amino_acids': []}

    for name_fragment in list_of_unique_name_fragments:
        data_dict1[name_fragment] = []
        data_dict2[name_fragment] = []

    # Process each TSV file
    for tsv_file in tsv_files_paths:
        # Check if the file name contains any value from list_of_unique_name_fragments
        if any(name_fragment in os.path.basename(tsv_file) for name_fragment in list_of_unique_name_fragments):
            sequence_wt, sequenc_mut, k_delta = read_sequence_column(tsv_file)

            # Append data to the dictionary
            data_dict1['WT_amino_acids'].append(sequence_wt)
            data_dict2['WT_amino_acids'].append(sequence_wt)

            # Find the matching name fragment and store the k_delta value
            for name_fragment in list_of_unique_name_fragments:
                if name_fragment in os.path.basename(tsv_file):
                    data_dict1[name_fragment].append(k_delta)
                    data_dict2[name_fragment].append(sequenc_mut)
                else:
                    data_dict1[name_fragment].append(float('nan'))
                    data_dict2[name_fragment].append(float('nan'))

    # Create the DataFrame
    df1 = pd.DataFrame(data_dict1)
    df1.set_index('WT_amino_acids', inplace=True)
    df2 = pd.DataFrame(data_dict2)
    df2.set_index('WT_amino_acids', inplace=True)
    
    df1.sort_values(by='WT_amino_acids', ascending=True, inplace=True)    
    df2.sort_values(by='WT_amino_acids', ascending=True, inplace=True)
    merged_df1 = df1.groupby('WT_amino_acids').max()
    
def custom_merge(series):
    return series.apply(lambda x: ', '.join(x.dropna().unique()) if x.notnull().any() else "NaN")

# Apply the custom_merge function to each group of rows with the same index ('WT_amino_acids')
merged_df2 = df2.groupby('WT_amino_acids').apply(custom_merge)

print(merged_df1)
print(merged_df2)


                    2bcx      6xxf      6y4o
WT_amino_acids                              
A11=glu         0.000000  0.485841       NaN
A14=glu         0.231432  0.267061       NaN
A15=ala         0.000000  1.990904       NaN
A18=leu         0.000000  0.144689       NaN
A19=phe         0.000000  0.000000  0.000000
A27=ile         0.008724  0.000000  0.206819
A32=leu         0.015981  0.000000  0.000000
A35=val         0.000000  2.673205  0.325599
A36=met         0.000000  0.000000  0.000000
A38=ser         3.471798  4.589735  3.897585
A39=leu         0.000000  0.000000  3.141124
A41=gln         3.167038  0.289900  2.593957
A47=glu         0.000000       NaN       NaN
A51=met         0.000000  0.627191  0.000000
A52=ile         0.161253  0.000000  0.063553
A54=glu         0.926277  4.178508       NaN
A55=val         0.000000  0.000000  0.000000
A63=ile         0.113335  0.000000  0.289148
A68=phe         0.000000  0.000000  0.000000
A71=met         0.000000  0.000000  0.000000
A72=met   